In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from glob import glob
import os
import numpy as np

In [2]:
files = pd.Series(glob('intron_events/*'))
files.head()

0    intron_events/HG00273.3.M_120202_7.intron_even...
1    intron_events/HG00112.6.M_120119_2.intron_even...
2    intron_events/HG00262.2.M_111215_8.intron_even...
3    intron_events/NA12004.4.M_120208_5.intron_even...
4    intron_events/NA20528.2.M_120131_1.intron_even...
dtype: object

In [3]:
def explore_filename(f):
    assay_name = os.path.basename(f).replace('.intron_events.csv', '')
    person_id = assay_name.split('.')[0]
    lab_id = assay_name.split('.')[1]
    complement_id = assay_name.split('.')[2]
    return dict(assay_name=assay_name, person_id=person_id, lab_id=lab_id,
                complement_id=complement_id)

def read_csv(filepath):
    df = pd.read_csv(filepath, sep='\t', header=None, low_memory=False, engine='c')
    df.rename(columns={0:'gene', 1:'intron', 2:'nread0', 3:'nread1'}, inplace=True)
    df['intron'] = df['intron'].apply(lambda v: int(v.split('_')[2]))
    ef = explore_filename(filepath)
    df['assay'] = ef['assay_name']
    df['person_id'] = ef['person_id']
    df['lab_id'] = ef['lab_id']
    df['lab_id'] = pd.to_numeric(df['lab_id'])
    df['nsuccesses'] = df['nread0']
    df['ntrials'] = df['nread0'] + df['nread1']
    del df['nread0'], df['nread1']
    return df

In [4]:
import multiprocessing
pool = multiprocessing.Pool(20)
df = pd.concat(pool.map(read_csv, files))
df.set_index(['assay', 'gene', 'intron']).head()

person_id  lab_id  nsuccesses  \
assay                gene            intron                                 
HG00273.3.M_120202_7 ENSG00000000003 1        HG00273       3           0   
                                     2        HG00273       3           0   
                                     3        HG00273       3           0   
                                     4        HG00273       3           0   
                                     5        HG00273       3           0   

                                             ntrials  
assay                gene            intron           
HG00273.3.M_120202_7 ENSG00000000003 1             0  
                                     2             0  
                                     3             0  
                                     4             0  
                                     5             0

In [5]:
df.set_index(['assay', 'gene', 'intron']).describe()

,lab_id,nsuccesses,ntrials
count,"128,811,890.00","128,811,890.00","128,811,890.00"
mean,2.86,23.43,296.99
std,1.98,566.55,"2,798.22"
min,1.00,0.00,0.00
25%,1.00,0.00,0.00
50%,2.00,0.00,18.00
75%,4.00,4.00,154.00
max,7.00,"1,065,842.00","1,418,836.00"


In [6]:
df.groupby('person_id').describe()

intron     lab_id  nsuccesses    ntrials
person_id                                                   
HG00096   count 225,590.00 225,590.00  225,590.00 225,590.00
          mean       10.14       1.00       27.96     292.35
          std        14.30       0.00      417.78   2,536.30
          min         1.00       1.00        0.00       0.00
          25%         3.00       1.00        0.00       0.00
          50%         6.00       1.00        0.00      24.00
          75%        13.00       1.00        8.00     171.00
          max       363.00       1.00   94,737.00 533,529.00
HG00097   count 225,590.00 225,590.00  225,590.00 225,590.00
          mean       10.14       7.00       40.71     467.09
          std        14.30       0.00      672.84   3,857.22
          min         1.00       7.00        0.00       0.00
          25%         3.00       7.00        0.00       0.00
          50%         6.00       7.00        1.00      39.00
          75%        13.00       7.00       12.00     292.00
          max       363.00       7.00  177,822.00 613,442.00
HG00099   count 451,180.00 451,180.00  451,180.00 451,180.00
          mean       10.14       3.00       17.45     238.64
          std        14.30       2.00      348.36   1,871.00
          min         1.00       1.00        0.00       0.00
          25%         3.00       1.00        0.00       0.00
          50%         6.00       3.00        0.00      18.00
          75%        13.00       5.00        4.00     148.00
          max       363.00       5.00   88,611.00 355,792.00
HG00100   count 225,590.00 225,590.00  225,590.00 225,590.00
          mean       10.14       2.00       19.18     250.76
          std        14.30       0.00      386.24   2,432.60
          min         1.00       2.00        0.00       0.00
          25%         3.00       2.00        0.00       0.00
          50%         6.00       2.00        0.00      17.00
...                    ...        ...         ...        ...
NA20816   std        14.30       0.00      493.15   2,370.20
          min         1.00       3.00        0.00       0.00
          25%         3.00       3.00        0.00       0.00
          50%         6.00       3.00        0.00      20.00
          75%        13.00       3.00        4.00     179.00
          max       363.00       3.00  155,533.00 349,284.00
NA20819   count 225,590.00 225,590.00  225,590.00 225,590.00
          mean       10.14       3.00       22.09     231.86
          std        14.30       0.00      501.05   2,134.96
          min         1.00       3.00        0.00       0.00
          25%         3.00       3.00        0.00       0.00
          50%         6.00       3.00        0.00      16.00
          75%        13.00       3.00        6.00     125.00
          max       363.00       3.00  196,397.00 290,231.00
NA20826   count 225,590.00 225,590.00  225,590.00 225,590.00
          mean       10.14       1.00       22.01     301.89
          std        14.30       0.00      461.76   2,614.09
          min         1.00       1.00        0.00       0.00
          25%         3.00       1.00        0.00       0.00
          50%         6.00       1.00        0.00      18.00
          75%        13.00       1.00        4.00     156.00
          max       363.00       1.00  114,816.00 541,212.00
NA20828   count 225,590.00 225,590.00  225,590.00 225,590.00
          mean       10.14       2.00       38.53     481.41
          std        14.30       0.00      876.61   4,378.93
          min         1.00       2.00        0.00       0.00
          25%         3.00       2.00        0.00       0.00
          50%         6.00       2.00        0.00      24.00
          75%        13.00       2.00        7.00     215.00
          max       363.00       2.00  264,247.00 622,381.00

[3712 rows x 4 columns]

In [9]:
df.head()

assay person_id  lab_id  nsuccesses  \
gene            intron                                                       
ENSG00000000003 1       HG00273.3.M_120202_7   HG00273       3           0   
                2       HG00273.3.M_120202_7   HG00273       3           0   
                3       HG00273.3.M_120202_7   HG00273       3           0   
                4       HG00273.3.M_120202_7   HG00273       3           0   
                5       HG00273.3.M_120202_7   HG00273       3           0   

                        ntrials  
gene            intron           
ENSG00000000003 1             0  
                2             0  
                3             0  
                4             0  
                5             0

In [ ]:
df.index